# Parallelization Examples for MD Simulation

This notebook demonstrates various parallelization techniques that can be applied to extend the 2-particle simulation to N particles.

**Examples covered:**
1. NumPy Vectorization
2. Numba JIT Compilation
3. Ensemble Parallelism
4. GPU Acceleration (PyTorch)
5. MPI (Message Passing Interface)
6. Async I/O (History Recording)

---

## Setup: Import Required Libraries

In [1]:
import numpy as np
import time
from typing import Tuple

print("✅ Core libraries imported successfully!")

✅ Core libraries imported successfully!


---

## Example 1: NumPy Vectorization

**Concept:** Replace Python loops with NumPy array operations for massive speedups.

**Expected speedup:** 2-10x  
**Difficulty:** ⭐ Easy  
**Best for:** Small to medium systems (N < 10,000)

In [2]:
def update_positions_loop(positions, velocities, dt, N):
    """Traditional loop-based position update (SLOW)."""
    for i in range(N):
        positions[i] += velocities[i] * dt
    return positions


def update_positions_vectorized(positions, velocities, dt):
    """Vectorized position update using NumPy (FAST)."""
    return positions + velocities * dt


def benchmark_vectorization():
    """Compare loop vs vectorized performance."""
    N = 10000
    positions = np.random.rand(N, 2) * 20.0
    velocities = np.random.rand(N, 2) * 0.1
    dt = 0.001
    
    # Loop version
    start = time.time()
    for _ in range(100):
        positions_loop = update_positions_loop(positions.copy(), velocities, dt, N)
    loop_time = time.time() - start
    
    # Vectorized version
    start = time.time()
    for _ in range(100):
        positions_vec = update_positions_vectorized(positions.copy(), velocities, dt)
    vec_time = time.time() - start
    
    print(f"Loop version: {loop_time:.4f}s")
    print(f"Vectorized version: {vec_time:.4f}s")
    print(f"Speedup: {loop_time/vec_time:.1f}x")


# Run benchmark
print("Running NumPy Vectorization Benchmark...")
print("=" * 60)
benchmark_vectorization()

Running NumPy Vectorization Benchmark...
Loop version: 2.3510s
Vectorized version: 0.0018s
Speedup: 1298.5x


---

## Example 2: Numba JIT Compilation

**Concept:** Use Just-In-Time (JIT) compilation to compile Python code to machine code.

**Expected speedup:** 10-100x  
**Difficulty:** ⭐⭐ Medium  
**Best for:** CPU-bound computations with loops

**Installation:** `pip install numba`

In [ ]:
try:
    from numba import jit, prange
    
    @jit(nopython=True)
    def compute_lj_force_numba(r_vec, epsilon=1.0, sigma=1.0):
        """Compute Lennard-Jones force (Numba-optimized)."""
        r = np.sqrt(r_vec[0]**2 + r_vec[1]**2)
        if r < 1e-10:
            return np.zeros(2)
        
        sr6 = (sigma / r) ** 6
        force_mag = 24.0 * epsilon / r * (2.0 * sr6**2 - sr6)
        r_hat = r_vec / r
        return force_mag * r_hat
    
    @jit(nopython=True, parallel=True)
    def compute_all_forces_parallel(positions, N, epsilon=1.0, sigma=1.0):
        """Compute all pairwise forces in parallel."""
        forces = np.zeros_like(positions)
        
        # Parallel loop over particles
        for i in prange(N):
            for j in range(i+1, N):
                r_vec = positions[i] - positions[j]
                f = compute_lj_force_numba(r_vec, epsilon, sigma)
                forces[i] += f
                forces[j] -= f
        
        return forces
    
    def benchmark_numba():
        """Benchmark Numba parallel force calculation."""
        N = 1000
        positions = np.random.rand(N, 2) * 20.0
        
        # Warm-up (JIT compilation happens here)
        print("Warming up (JIT compilation)...")
        _ = compute_all_forces_parallel(positions, N)
        
        # Benchmark
        print("Running benchmark...")
        start = time.time()
        for _ in range(10):
            forces = compute_all_forces_parallel(positions, N)
        numba_time = time.time() - start
        
        print(f"Numba parallel force calculation: {numba_time:.4f}s for {N} particles")
        print(f"Time per iteration: {numba_time/10:.4f}s")
    
    print("Running Numba JIT Compilation Benchmark...")
    print("=" * 60)
    benchmark_numba()
    
except ImportError:
    print("❌ Numba not available.")
    print("Install with: pip install numba")

---

## Example 3: Ensemble Parallelism

**Concept:** Run multiple independent simulations in parallel (e.g., parameter sweeps).

**Expected speedup:** Nx (N = number of cores)  
**Difficulty:** ⭐ Easy  
**Best for:** Parameter studies, replica exchange

In [ ]:
def run_single_simulation(params):
    """Run a single simulation with given parameters."""
    temperature, n_steps = params
    
    # Simulate some work
    np.random.seed(int(temperature * 1000))
    positions = np.random.rand(100, 2) * 20.0
    velocities = np.random.rand(100, 2) * 0.1 * temperature
    
    # Simple integration
    dt = 0.001
    for _ in range(n_steps):
        positions += velocities * dt
    
    # Return some result
    avg_position = np.mean(positions)
    return temperature, avg_position


def run_ensemble_serial(temperatures, n_steps):
    """Run ensemble of simulations serially."""
    results = []
    for temp in temperatures:
        result = run_single_simulation((temp, n_steps))
        results.append(result)
    return results


def run_ensemble_parallel(temperatures, n_steps):
    """Run ensemble of simulations in parallel."""
    from multiprocessing import Pool
    
    params = [(temp, n_steps) for temp in temperatures]
    
    with Pool(processes=4) as pool:
        results = pool.map(run_single_simulation, params)
    
    return results


def benchmark_ensemble():
    """Compare serial vs parallel ensemble runs."""
    temperatures = [100, 200, 300, 400, 500, 600, 700, 800]
    n_steps = 1000
    
    # Serial
    start = time.time()
    results_serial = run_ensemble_serial(temperatures, n_steps)
    serial_time = time.time() - start
    
    # Parallel
    start = time.time()
    results_parallel = run_ensemble_parallel(temperatures, n_steps)
    parallel_time = time.time() - start
    
    print(f"Serial ensemble: {serial_time:.4f}s")
    print(f"Parallel ensemble: {parallel_time:.4f}s")
    print(f"Speedup: {serial_time/parallel_time:.1f}x")
    print(f"\nResults (Temperature, Avg Position):")
    for temp, avg_pos in results_parallel[:3]:
        print(f"  T={temp}K: {avg_pos:.4f}")


# Run benchmark
print("Running Ensemble Parallelism Benchmark...")
print("=" * 60)
benchmark_ensemble()

Running Ensemble Parallelism Benchmark...


---

## Example 4: GPU Acceleration with PyTorch

**Concept:** Offload computations to GPU for massive parallelism.

**Expected speedup:** 50-500x  
**Difficulty:** ⭐⭐⭐ Hard  
**Best for:** Large systems (N > 10,000), long simulations

**Installation:** `pip install torch`

In [ ]:
try:
    import torch
    
    def update_positions_gpu(positions, velocities, dt):
        """Update positions on GPU using PyTorch."""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        # Move to GPU
        pos_gpu = torch.tensor(positions, device=device, dtype=torch.float32)
        vel_gpu = torch.tensor(velocities, device=device, dtype=torch.float32)
        
        # Compute on GPU
        pos_gpu += vel_gpu * dt
        
        # Move back to CPU
        return pos_gpu.cpu().numpy()
    
    def benchmark_gpu():
        """Benchmark GPU vs CPU for position updates."""
        N = 100000  # Large number for GPU to shine
        positions = np.random.rand(N, 2).astype(np.float32) * 20.0
        velocities = np.random.rand(N, 2).astype(np.float32) * 0.1
        dt = 0.001
        
        if not torch.cuda.is_available():
            print("⚠️  CUDA not available. Running on CPU only.")
            print("GPU benchmarks will not show speedup without a CUDA-capable GPU.\n")
            device = torch.device('cpu')
        else:
            device = torch.device('cuda')
            print(f"✅ Using GPU: {torch.cuda.get_device_name(0)}\n")
        
        # CPU version
        start = time.time()
        for _ in range(100):
            pos_cpu = update_positions_vectorized(positions.copy(), velocities, dt)
        cpu_time = time.time() - start
        
        # GPU version
        pos_gpu = torch.tensor(positions, device=device)
        vel_gpu = torch.tensor(velocities, device=device)
        
        # Warm-up
        _ = pos_gpu + vel_gpu * dt
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        
        start = time.time()
        for _ in range(100):
            pos_gpu += vel_gpu * dt
        if torch.cuda.is_available():
            torch.cuda.synchronize()
        gpu_time = time.time() - start
        
        print(f"CPU (NumPy): {cpu_time:.4f}s")
        print(f"GPU (PyTorch): {gpu_time:.4f}s")
        if torch.cuda.is_available():
            print(f"Speedup: {cpu_time/gpu_time:.1f}x")
        else:
            print("(No GPU speedup - running on CPU)")
    
    print("Running GPU Acceleration Benchmark...")
    print("=" * 60)
    benchmark_gpu()
    
except ImportError:
    print("❌ PyTorch not available.")
    print("Install with: pip install torch")

---

## Example 5: MPI (Message Passing Interface)

**Concept:** Distribute computation across multiple nodes in a cluster.

**Expected speedup:** 10-1000x (scales to 1000s of cores)  
**Difficulty:** ⭐⭐⭐⭐⭐ Expert  
**Best for:** Very large systems (N > 100,000), HPC clusters

**Installation:** Requires MPI installation + `pip install mpi4py`

**Note:** MPI programs must be run from the command line with `mpiexec`, not in Jupyter notebooks. This cell will generate an example MPI script for you.

In [ ]:
MPI_EXAMPLE_CODE = '''
"""MPI Example - Save this as mpi_example.py and run with:
    mpiexec -n 4 python mpi_example.py

This demonstrates basic MPI concepts for MD simulations.
"""

from mpi4py import MPI
import numpy as np
import time

def mpi_hello_world():
    """Basic MPI: Each process prints its rank."""
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    print(f"Hello from rank {rank} of {size} processes")
    return rank, size

def mpi_parallel_sum():
    """Demonstrate MPI reduction (sum across all processes)."""
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    local_value = rank * 10
    total = comm.allreduce(local_value, op=MPI.SUM)
    if rank == 0:
        print(f"\\nMPI Reduction: Sum of all ranks = {total}")
    return total

def mpi_domain_decomposition():
    """Simulate domain decomposition for MD."""
    comm = MPI.COMM_WORLD
    rank = comm.Get_rank()
    size = comm.Get_size()
    
    total_particles = 1000
    local_n = total_particles // size
    
    np.random.seed(rank)
    local_positions = np.random.rand(local_n, 3) * 20.0
    local_velocities = np.random.rand(local_n, 3) * 0.1
    
    dt = 0.001
    start = time.time()
    for _ in range(100):
        local_positions += local_velocities * dt
    elapsed = time.time() - start
    
    all_times = comm.gather(elapsed, root=0)
    
    if rank == 0:
        print(f"\\nMPI Domain Decomposition:")
        print(f"  Total particles: {total_particles}")
        print(f"  Particles per rank: {local_n}")
        print(f"  Average time: {np.mean(all_times):.4f}s")

if __name__ == "__main__":
    rank, size = mpi_hello_world()
    comm = MPI.COMM_WORLD
    comm.Barrier()
    
    if rank == 0:
        print("\\n" + "=" * 60)
    
    mpi_parallel_sum()
    comm.Barrier()
    
    if rank == 0:
        print("=" * 60)
    
    mpi_domain_decomposition()
    
    if rank == 0:
        print("=" * 60)
        print("\\nMPI examples complete!")
        print("\\nTo run: mpiexec -n 4 python mpi_example.py")
'''

def show_mpi_info():
    """Show information about MPI and how to use it."""
    print("MPI (Message Passing Interface) Information")
    print("=" * 60)
    
    try:
        from mpi4py import MPI
        print("✅ mpi4py is installed!")
        print(f"   MPI Version: {MPI.Get_version()}")
        
        # Save example code
        with open('mpi_example.py', 'w') as f:
            f.write(MPI_EXAMPLE_CODE)
        
        print("\n📝 MPI example saved to: mpi_example.py")
        print("\n🚀 To run MPI example:")
        print("   mpiexec -n 4 python mpi_example.py")
        print("\n💡 This will run 4 parallel processes")
        
    except ImportError:
        print("❌ mpi4py not installed")
        print("\n📦 To install MPI support:")
        print("\n   On Linux/Mac:")
        print("     sudo apt-get install libopenmpi-dev  # Ubuntu/Debian")
        print("     brew install open-mpi               # macOS")
        print("     pip install mpi4py")
        print("\n   On Windows:")
        print("     1. Download MS-MPI from Microsoft")
        print("     2. Install MS-MPI SDK and Runtime")
        print("     3. pip install mpi4py")
        print("\n   Or use conda:")
        print("     conda install -c conda-forge mpi4py")

# Show MPI information
show_mpi_info()

---

## Example 6: Async I/O (History Recording)

**Concept:** Use asynchronous I/O to write trajectory data without blocking computation.

**Expected speedup:** Reduces I/O bottlenecks  
**Difficulty:** ⭐⭐ Medium  
**Best for:** Large trajectory files, concurrent I/O operations

**Installation:** `pip install aiofiles`

In [ ]:
try:
    import asyncio
    import aiofiles
    
    def save_history_sync(filename, positions, velocities, n_steps):
        """Save trajectory history synchronously (BLOCKING)."""
        with open(filename, 'w') as f:
            for step in range(n_steps):
                # Simulate position update
                positions += velocities * 0.001
                
                # Write to file (BLOCKS computation)
                f.write(f"Step {step}\n")
                for i, pos in enumerate(positions):
                    f.write(f"  Particle {i}: {pos[0]:.6f}, {pos[1]:.6f}\n")
    
    
    async def save_history_async(filename, positions, velocities, n_steps):
        """Save trajectory history asynchronously (NON-BLOCKING)."""
        async with aiofiles.open(filename, 'w') as f:
            for step in range(n_steps):
                # Simulate position update
                positions += velocities * 0.001
                
                # Write to file (NON-BLOCKING - allows other work)
                await f.write(f"Step {step}\n")
                for i, pos in enumerate(positions):
                    await f.write(f"  Particle {i}: {pos[0]:.6f}, {pos[1]:.6f}\n")
    
    
    def benchmark_async_io():
        """Compare synchronous vs asynchronous I/O."""
        N = 100
        n_steps = 100
        positions = np.random.rand(N, 2) * 20.0
        velocities = np.random.rand(N, 2) * 0.1
        
        # Synchronous I/O
        start = time.time()
        save_history_sync('trajectory_sync.txt', positions.copy(), velocities, n_steps)
        sync_time = time.time() - start
        
        # Asynchronous I/O
        start = time.time()
        asyncio.run(save_history_async('trajectory_async.txt', positions.copy(), velocities, n_steps))
        async_time = time.time() - start
        
        print(f"Synchronous I/O: {sync_time:.4f}s")
        print(f"Asynchronous I/O: {async_time:.4f}s")
        print(f"Speedup: {sync_time/async_time:.2f}x")
        print(f"\n💡 Async I/O shines when:")
        print(f"   - Writing large trajectory files")
        print(f"   - Doing other work while I/O happens")
        print(f"   - Multiple concurrent I/O operations")
        
        # Clean up
        import os
        try:
            os.remove('trajectory_sync.txt')
            os.remove('trajectory_async.txt')
            print("\n✅ Temporary files cleaned up")
        except:
            pass
    
    print("Running Async I/O Benchmark...")
    print("=" * 60)
    benchmark_async_io()
    
except ImportError:
    print("❌ aiofiles not available.")
    print("Install with: pip install aiofiles")

---

## Summary

### Performance Comparison

| Technique | Speedup | Difficulty | Best For |
|-----------|---------|------------|----------|
| NumPy Vectorization | 2-10x | ⭐ Easy | Small-medium N |
| Numba JIT | 10-100x | ⭐⭐ Medium | CPU-bound loops |
| Ensemble Parallelism | Nx | ⭐ Easy | Parameter studies |
| GPU (PyTorch) | 50-500x | ⭐⭐⭐ Hard | Large N, long runs |
| MPI | 10-1000x | ⭐⭐⭐⭐⭐ Expert | HPC clusters |
| Async I/O | Reduces I/O bottlenecks | ⭐⭐ Medium | Large trajectory files |

### Recommendations by System Size

- **N < 1,000:** NumPy vectorization
- **N = 1,000-10,000:** NumPy + Numba
- **N = 10,000-100,000:** GPU acceleration
- **N > 100,000:** MPI + GPU (HPC cluster)
- **Parameter sweeps:** Ensemble parallelism

### Next Steps

1. **Read:** [PARALLELIZATION_GUIDE.md](../PARALLELIZATION_GUIDE.md) for detailed theory
2. **Install:** Optional packages from [requirements.txt](../requirements.txt)
3. **Experiment:** Modify examples with different problem sizes
4. **Apply:** Implement techniques when extending to N-body
5. **Measure:** Always benchmark before and after!

---

*This notebook was created with Augment Agent assistance.*